In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import plotly.graph_objects as go 

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
def smooth_data(x, y, idx=20, estimator=DecisionTreeRegressor):
    fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers', name='Data', opacity=0.85)])
    fig.update_traces(marker_size=12, marker_line_width=1.5)
    rft = np.fft.rfft(y)
    
    X_train, X_test, y_train, y_test = train_test_split(x.reshape(-1, 1), y, test_size=0.85, random_state=12)
    model = estimator().fit(X_train, y_train)
    fig.add_traces(go.Scatter(x=x, y=model.predict(x.reshape(-1, 1)),
                              visible=False, opacity=0.45, name=f"Base Model", 
                              line=go.scatter.Line(dash='dot', width=2)))
    base_mse = mean_squared_error(y, model.predict(x.reshape(-1, 1))).round(3)
    errors_list = []
    for i in range(1, idx):
        new_rft = rft.copy()
        new_rft[i:] = 0 
        smooth_y = np.fft.irfft(new_rft)
        X_train, X_test, y_train, y_test = train_test_split(x.reshape(-1, 1), smooth_y, test_size=0.85, random_state=12)
        model = estimator().fit(X_train, y_train)
        pred = model.predict(x.reshape(-1, 1))
        fig.add_traces(go.Scatter(x=x, y=pred, visible=False, 
                                  name='Model<br>(smoothed data)', line=go.scatter.Line(width=3, dash='dashdot')))
        fig.add_traces(go.Scatter(x=x, y=smooth_y, visible=False, name='Smoothed data', 
                                  line=go.scatter.Line(width=3, dash='solid')))
        errors_list.append(mean_squared_error(y, pred).round(3))
    steps = []
    fig.data[1].visible = True
    fig.data[0].visible = True
    for i in range(1, idx - 1):
        step = dict(
            method="update",
            args=[{"visible": [True] * 2 + [False] * (len(fig.data) - 1)},
                  {"title": f"Синусоид {i-1} <br>MSE: {errors_list[i - 1]} (smooth)<br>MSE: {base_mse} (base)"}],  
            label=f"FFT: {i}",
        )
        step["args"][0]["visible"][2 * i] = True  
        step["args"][0]["visible"][2 * i + 1] = True  
        steps.append(step)
        sliders = [
            dict(
                currentvalue={"prefix": ""},
                pad={"t": 50},
                steps=steps
            ),
        ]
    
    fig.update_layout(
        sliders=sliders,
        title="Fourier Transformation",   
        autosize=False,
        width=1000,
        height=750,
    )
    
    fig.update_yaxes(range=[y.min() * 0.9, y.max() * 1.1])
    fig.show()

In [ ]:
x = np.linspace(0, 5, 284)
y = (lambda x: x ** 2 / 2 - x ** 3 / 6 + x ** 4 / 125)(x)

base_noise = 0.2
bonus_noise = 1 
noise = np.random.normal(0, base_noise, size=len(x))
bonus_noise = np.random.normal(0, bonus_noise, size=len(x)) * np.random.choice([0, 1], size=len(x), p=[0.75, 0.25])
y += noise
y += bonus_noise

In [ ]:
smooth_data(x, y, idx=int(len(y) ** 0.9), estimator=GradientBoostingRegressor)

In [ ]:
def ft(x):
    N = len(x)
    return np.array([sum(x * np.exp(-2j * np.pi / N * k * np.arange(len(x)))) for k in range(N)])

def ift(x):
    N = len(x)
    return np.array([sum(x * np.exp(2j * np.pi / N * n * np.arange(len(x)))) for n in range(N)]) / N